In [71]:
from datetime import datetime, timedelta, date
import calendar
import holidays

nerc_holidays = ["New Year's Day", "Memorial Day", "Independence Day",
                 "Labor Day", "Thanksgiving", "Christmas Day"]     # define NERC holidays

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']     

In [279]:
def get_hours_daily(iso, peak_type, date, eastern):
    """
    returns the number of hours of one calendar day.
    """
    
    result = 0
    
    # weekend
    weekend = False
    if date.weekday() > 4 and eastern:
        weekend = True    #Western market takes Saturday as a weekday.
    if date.weekday() > 5 and not eastern:
        weekend = True
            
    # holiday
    holiday = False
    if holidays.UnitedStates().get(date) and holidays.UnitedStates().get(date) in nerc_holidays:
        holiday = True
            
    # DST: if there's a Daylight-saving offset.
    # Not in the 'MISO', -1 if it's the second Sunday of March and +1 the first Sunday of November.
    DST = 0
    if iso != 'MISO' and date.weekday() == 6:
        if date.month == 3 and (date - timedelta(days=7)).month == 3 and (date - timedelta(days=14)).month == 2:
            DST = -1
        elif date.month == 11 and (date - timedelta(days=7)).month == 10:
            DST = 1
            
            
    # peak = flat
    if peak_type == 'flat':
        result = 24 + DST
        
    # peak = onpeak
    if peak_type == 'onpeak':
        if (not holiday) and (not weekend):
            result = 16
        else:
            result = 0
        
    # peak = offpeak
    if peak_type == 'offpeak':
        if (not holiday) and (not weekend):
            result = 8 + DST
        else:
            result = 24 + DST
        
    # peak = 2x16H
    if peak_type == '2x16H':
        if holiday or weekend:
            result = 16
        else:
            result = 0
        
    # peak = 7x8
    if peak_type == '7x8':
        result = 8 + DST

    # mark the peak days
    if (not holiday) and (not weekend):
        peakday = True
    else:
        peakday = False
        
    return (result, peakday)

In [280]:
def get_hours_monthly(iso, peak_type, year, month, eastern):
    """
    returns the number of hours of one month.
    """
    
    result = {}
    num_hour = 0
    num_peakday = 0
    
    num_days = calendar.monthrange(year, month)[1]
    days = [datetime(year, month, day) for day in range(1, num_days+1)]
    
    # add up the number of hours of each day in the month
    for day in days:
        temp_result = get_hours_daily(iso, peak_type, day, eastern)
        num_hour += temp_result[0]     
        num_peakday += temp_result[1]
        
    result['num.hour'] = num_hour
    result['start.date'] = days[0].strftime('%Y-%m-%d')
    result['end.date'] = days[-1].strftime('%Y-%m-%d')  
    result['total.days'] = len(days)
    result['peak.days'] = num_peakday
    
    return result

In [289]:
def get_hours(iso, peak_type, period):
    """
    returns number of hours by iso/peak.type/period
    """
    
    result = {'iso': iso, 
              'peak.type': peak_type}
    peak_days = 0
    
    # eastern / western iso
    eastern = False
    if iso in ['PJMISO', 'MISO', 'ERCOT', 'SPPISO', 'NYISO']:
        eastern = True

    
    # daily
    if period.find('-') != -1:
        date = datetime.strptime(period, '%Y-%m-%d')
        result['start.date'] = date.strftime('%Y-%m-%d')
        result['end.date'] = date.strftime('%Y-%m-%d')
        temp_result = get_hours_daily(iso, peak_type, date, eastern)
        result['num.hour'] = temp_result[0]
        peak_days = temp_result[1]
            
            
    # monthly: 2018Mar
    if any(ele in period for ele in months):
        year = int(period[:4])
        for m in months:
            if m == period[4:]:
                month = months.index(m) + 1
        temp_result = get_hours_monthly(iso, peak_type, year, month, eastern)
        result['num.hour'] = temp_result['num.hour']
        result['start.date'] = temp_result['start.date']
        result['end.date'] = temp_result['end.date']
        peak_days = temp_result['peak.days']
        
    
    # quarterly: 2018Q2
    if period.find('Q') != -1:
        num_hour = 0
        year = int(period[:4])
        quarter = int(period[-1])
        for month in range(quarter*3-2,quarter*3+1):   # add up the number of hours of each month in the quarter
            temp_result = get_hours_monthly(iso, peak_type, year, month, eastern)
            num_hour += temp_result['num.hour']
            peak_days += temp_result['peak.days']
            if month == (quarter*3-2):
                startdate = temp_result['start.date']
            if month == quarter*3:
                enddate = temp_result['end.date']
        result['num.hour'] = num_hour
        result['start.date'] = startdate
        result['end.date'] = enddate
        
    
    # yearly: 2018A
    if period[-1] == 'A':
        num_hour = 0
        year = int(period[:4])
        for month in range(1, 13):
            temp_result = get_hours_monthly(iso, peak_type, year, month, eastern)
            num_hour += temp_result['num.hour']
            peak_days += temp_result['peak.days']
        result['num.hour'] = num_hour
        result['start.date'] = "%s-01-01"%(year)
        result['end.date'] = "%s-12-31"%(year)
 
        
    return result

### Testing

In [290]:
# test daily

date = datetime.strptime('2022-1-31', '%Y-%m-%d')
get_hours_daily("ERCOT", "onpeak", date, True)

(16, True)

In [291]:
# test monthly

# get_hours("ERCOT", "offpeak", '2022-11-6')
get_hours("CASIO", "onpeak", '2022Dec')

{'iso': 'CASIO',
 'peak.type': 'onpeak',
 'num.hour': 432,
 'start.date': '2022-12-01',
 'end.date': '2022-12-31'}

In [292]:
# test quarterly

get_hours("CASIO", "offpeak", '2023Q1')

{'iso': 'CASIO',
 'peak.type': 'offpeak',
 'num.hour': 927,
 'start.date': '2023-01-01',
 'end.date': '2023-03-31'}

In [293]:
# test annually

get_hours("CASIO", "offpeak", '2023A')

{'iso': 'CASIO',
 'peak.type': 'offpeak',
 'num.hour': 3848,
 'start.date': '2023-01-01',
 'end.date': '2023-12-31'}

references: https://www.nerc.com/comm/OC/RS%20Agendas%20Highlights%20and%20Minutes%20DL/Additional_Off-peak_Days.pdf